In [ ]:
import pandas as pd
%load_ext autoreload
%autoreload 2

In [1]:
from data.election_results_ingest import IngestElectionResults
from elections.elections import Elections
from pathlib import Path
import geopandas as gpd
import folium

In [2]:
ier = IngestElectionResults('~/Documents/data')

In [4]:
ier.ingest('../resources/cobb_20220524.xml')

In [7]:
e = Elections('~/Documents/data')
df = e.get_results('2022-05-24', contest_pattern=r'STATE HOUSE\b.*\b40\b')

In [8]:
df

,choice,party,county,precinct_name,votes
0,DOUG STONER,DEM,COBB,LINDLEY 01,30
1,DOUG STONER,DEM,COBB,MABLETON 03,0
2,DOUG STONER,DEM,COBB,MABLETON 04,0
3,DOUG STONER,DEM,COBB,NICKAJACK 01,368
4,DOUG STONER,DEM,COBB,NORTON PARK 01,0
5,DOUG STONER,DEM,COBB,OAKDALE 01,314
6,DOUG STONER,DEM,COBB,SMYRNA 1A,53
7,DOUG STONER,DEM,COBB,SMYRNA 4A,453
8,DOUG STONER,DEM,COBB,SMYRNA 6A,408
9,DOUG STONER,DEM,COBB,SMYRNA 7A,425


In [9]:
df1 = df[['party', 'county', 'precinct_name', 'votes']].groupby(['party', 'county', 'precinct_name']).sum().reset_index()

In [10]:
df1

,party,county,precinct_name,votes
0,DEM,COBB,LINDLEY 01,56
1,DEM,COBB,MABLETON 03,0
2,DEM,COBB,MABLETON 04,0
3,DEM,COBB,NICKAJACK 01,712
4,DEM,COBB,NORTON PARK 01,0
5,DEM,COBB,OAKDALE 01,497
6,DEM,COBB,SMYRNA 1A,95
7,DEM,COBB,SMYRNA 4A,664
8,DEM,COBB,SMYRNA 6A,598
9,DEM,COBB,SMYRNA 7A,696


In [13]:
stats = df1.pivot(index=['precinct_name', 'county'], columns='party', values='votes').reset_index()
stats = stats.assign(share=stats.loc[:, 'REP']/(stats.loc[:, 'REP']+stats.loc[:, 'DEM'])*100)
stats = stats[stats.share.notna()].reset_index()

In [14]:
stats

party,index,precinct_name,county,DEM,REP,share
0,0,LINDLEY 01,COBB,56,23,29.113924
1,3,NICKAJACK 01,COBB,712,201,22.015334
2,5,OAKDALE 01,COBB,497,445,47.239915
3,6,SMYRNA 1A,COBB,95,94,49.735450
4,7,SMYRNA 4A,COBB,664,584,46.794872
5,8,SMYRNA 6A,COBB,598,546,47.727273
6,9,SMYRNA 7A,COBB,696,281,28.761515
7,10,VININGS 01,COBB,362,331,47.763348
8,11,VININGS 02,COBB,707,566,44.461901
9,12,VININGS 03,COBB,540,452,45.564516


In [21]:
p = Path('~/Documents/data/ga/politics/maps/vtd/VTDlatest-Shapefile/VTD2020-Shapefile.shp').expanduser()
precincts = gpd.read_file(p)[['ID', 'PRECINCT_N', 'CTYNAME', 'geometry']].to_crs(epsg=3035)
precincts.columns = ['id', 'precinct_name', 'county', 'geometry']
precinct_stats = precincts.merge(stats, on=['precinct_name', 'county'], how='inner').drop(columns=['index'])

In [22]:
precinct_stats

,id,precinct_name,county,geometry,DEM,REP,share
0,117249,VININGS 01,COBB,"POLYGON ((-1999132.171 6210921.102, -1999189.3...",362,331,47.763348
1,128770,LINDLEY 01,COBB,"MULTIPOLYGON (((-2006905.047 6211272.483, -200...",56,23,29.113924
2,132097,SMYRNA 7A,COBB,"MULTIPOLYGON (((-2003823.758 6215192.751, -200...",696,281,28.761515
3,137217,SMYRNA 4A,COBB,"POLYGON ((-2002698.759 6219152.038, -2002617.2...",664,584,46.794872
4,138500,VININGS 04,COBB,"POLYGON ((-2004029.889 6209728.024, -2003973.2...",300,730,70.873786
5,140802,SMYRNA 1A,COBB,"MULTIPOLYGON (((-1999392.278 6214182.270, -199...",95,94,49.735450
6,141057,SMYRNA 6A,COBB,"POLYGON ((-2002108.169 6213926.454, -2002286.4...",598,546,47.727273
7,141058,VININGS 03,COBB,"POLYGON ((-2002851.009 6213148.399, -2002837.6...",540,452,45.564516
8,141059,VININGS 02,COBB,"MULTIPOLYGON (((-2001054.094 6214072.344, -200...",707,566,44.461901
9,141313,OAKDALE 01,COBB,"MULTIPOLYGON (((-2002108.169 6213926.454, -200...",497,445,47.239915


In [23]:
p = Path('~/Documents/data/ga/politics/maps/hse-prop1-2021-shape/districts.shp').expanduser()
ga_hse_map = gpd.read_file(p).to_crs(epsg=3035)
ga_hse_40_map = ga_hse_map[ga_hse_map.DISTRICT == '040'].reset_index()
center = ga_hse_40_map.centroid.to_crs(epsg=4326)
precincts = precincts.overlay(ga_hse_40_map[['geometry']], how='intersection', keep_geom_type=False).to_crs(epsg=4326)
precinct_stats = precinct_stats.assign(key=precinct_stats.precinct_name+precinct_stats.county)

In [24]:
precinct_stats = precinct_stats.assign(share1=precinct_stats.share.apply(lambda x: f'{x:.1f}%'))
precinct_stats.columns = ['id', 'Precinct Name', 'county', 'geometry', 'DEM', 'REP', 'share', "key", 'REP Share']

In [29]:
m = folium.Map(location=[center[0].y, center[0].x], zoom_start=12)

cp = folium.Choropleth(
    geo_data=precinct_stats,
    name="choropleth",
    data=precinct_stats,
    columns=["key", "share"],
    key_on="feature.properties.key",
    fill_color="YlOrRd",
    fill_opacity=0.5,
    line_opacity=1,
    line_weight=1.5,
    line_color='darkgray',
    legend_name="Republican Share of Vote in District 40 (%)",
).add_to(m)

folium.GeoJsonTooltip(['Precinct Name', 'REP', "DEM", 'REP Share']).add_to(cp.geojson)
m

In [30]:
m.save("ga_house_40.html")